In [49]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt','r').read().splitlines()

In [3]:
chars = sorted(set(''.join(words)))

In [4]:
stoi = {}
itos = {}
i=1
for char in chars:
    stoi[char] = i
    i+=1
stoi["."] = 0
itos={i:s for s,i in stoi.items()}

In [27]:
block_size = 3
def buid_dataset(words):
    X, Y =[], []

    for word in  words + ['.']:
        context = [0] * block_size
        for l in word:
            ix = stoi[l]
            X.append(context)
            Y.append(ix)
            #Updating
            context=context[1:]+[ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X,Y
    
# Spliting To Train Val and Test
import random
random.shuffle(words)
N=len(words)
n1 = int(N * 0.8)
n2 = int(N * 0.9)
Xtrain , ytrain = buid_dataset(words[:n1])
Xval , yval = buid_dataset(words[n1:n2])
Xtest ,yYtest = buid_dataset(words[n2:])

torch.Size([156954, 3]) torch.Size([156954])
torch.Size([19639, 3]) torch.Size([19639])
torch.Size([19523, 3]) torch.Size([19523])


In [6]:
C = torch.randn((27,10))

### Training a Deep Neural Network

In [45]:
class Linear:
    
    def  __init__(self, fan_in, fan_out):
        self.weight = torch.randn((fan_in,fan_out)) / fan_in**0.5
        #self.bias = torch.zeros(fan_out)
    
    def __call__(self, x):
        self.out = x @ self.weight #+ self.bias
        return self.out
    def parameters(self):
        return [self.weight]

class BatchNorm1d:
    
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        # parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        # buffers (trained with a momentum update)
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
        
    def __call__(self, x):
        if self.training:
            xmean = x.mean(0, keepdim = True)
            xvar = x.var(0, keepdim = True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        # update the buffers
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
    
class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    
    def parameters(self):
        return []

In [46]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 100 # the number of neurons in the hidden layer of the MLP
g = torch.Generator().manual_seed(2147483647) # for reproducibility
vocab_size = 27
n_embd =10

C = torch.randn((vocab_size, n_embd),            generator=g)
layers = [
  Linear(n_embd * block_size, n_hidden), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden), BatchNorm1d(n_hidden), Tanh(),
  Linear(           n_hidden, vocab_size), BatchNorm1d(vocab_size),
]

In [47]:
with torch.no_grad():
    # last layer: make less confident
    layers[-1].gamma *= 0.1
    #layers[-1].weight *= 0.1
    # all other layers: apply gain
    for layer in layers[:-1]:
        if isinstance(layer, Linear):
            layer.weight *= 1.0 #5/3

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad = True

47024


In [55]:
max_steps = 2000
batch_size = 32
lossi = []
for i in range(max_steps):
    # minibatch construct
    ix = torch.randint(0, Xtrain.shape[0], (batch_size,))
    Xb, Yb = Xtrain[ix], ytrain[ix]
    
    # forward pass
    emb = C[Xb]
    x = emb.view(emb.shape[0], -1)
    for layer in layers:
        #print(f"Before layer {i}: {layer.__class__.__name__}, x shape: {None if x is None else x.shape}")
        x = layer(x)
        #print(f"After layer {i}: x shape: {None if x is None else x.shape}")
    loss = F.cross_entropy(x, Yb)
    
    # backward pass
    for layer in layers:
        layer.out.retain_grad() # AFTER_DEBUG: would take out retain_graph
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # update
    lr = 0.1 if i < 150000 else 0.01 # step learning rate decay
    for p in parameters:
        p.data += -lr * p.grad
        
    # track stats
    if i % 10000 == 0: # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())
    #with torch.no_grad():
        #ud.append([((lr*p.grad).std() / p.data.std()).log10().item() for p in parameters])

      0/   2000: 2.3139
